### <font color='darkred'>IA Aplicado a Finanças</font>
### <font color='darkgreen'>Prever inadimplência com base em Análise de Crédito</font>   
### <font color='darkblue'>Parte 1 - Deploy do Modelo</font>   

### <font color='darkred'>Instalando e Carregando Pacotes</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Imports
import pandas as pd
import torch
import torch.nn as nn
from joblib import load
import warnings
warnings.filterwarnings('ignore')

In [3]:
### Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Michelle Bouhid" --iversions

Author: Michelle Bouhid

platform: 1.0.8
torch   : 2.5.1
joblib  : 1.4.2
pandas  : 1.5.3



In [4]:
# Definição da arquitetura do modelo (deve ser a mesma que foi treinada)
# Cria a casta e vai preencher com o conteudo do arquivo criado no treino/teste: michelle_modelo.pth
class ModeloNN(nn.Module):
    def __init__(self, input_dim):
        super(ModeloNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

In [5]:
# Carregar os dados de entrada - novos dados
data = pd.read_csv('dados/novos_clientes.csv')

# Carregar o scaler => tudo que fizer no treino e teste e aos novos dados
scaler = load('modelos/michelle_scaler.pkl')

# Aplicar a transformação do scaler nos novos dados
X_new = scaler.transform(data)

# Converter os dados transformados para tensores PyTorch
X_new_tensor = torch.FloatTensor(X_new)

# Definir a arquitetura do modelo
# o modelo espera receber o shape do novos dados, no caso nº de calunas
model = ModeloNN(X_new.shape[1])

# Carregar o modelo treinado
model.load_state_dict(torch.load('modelos/michelle_modelo.pth'))



<All keys matched successfully>

In [6]:
# Definir o modelo como modo de avaliação (importante para desativar camadas como Dropout, se existirem)
# eval = avaliation
model.eval()

# Realizar as previsões
with torch.no_grad():  # desativar o cálculo do gradiente para melhorar a performance
    probabilities = model(X_new_tensor) #parametro novos dados em formato de tensor
    
# Convertendo as probabilidades para classes com base no limiar de 0.5
# o modelo entrega um valor de probabilidade, se tiver maior que 0.5 entrega 1, vai ficar inadimplente
# poderia melhorar o codigo entregando o valor da probabilidade que o modelo calculou!
predicted_classes = (probabilities > 0.5).int().numpy()

print('\nPrevisões Feitas Pelo Modelo de IA:\n')




Previsões Feitas Pelo Modelo de IA:



In [7]:
# Loop para imprimir a previsão para cada cliente
for idx, prediction in enumerate(predicted_classes):
    if prediction == 0:
        print(f'Este cliente: {data.iloc[idx].to_dict()} provavelmente não será inadimplente.')
    else:
        print(f'Este cliente: {data.iloc[idx].to_dict()} provavelmente será inadimplente.')



Este cliente: {'idade': 93.0, 'salario': 9302.87, 'historico_credito': 0.53, 'emprestimos_ativos': 9.0} provavelmente não será inadimplente.
Este cliente: {'idade': 62.0, 'salario': 10953.85, 'historico_credito': 0.96, 'emprestimos_ativos': 4.0} provavelmente não será inadimplente.
Este cliente: {'idade': 35.0, 'salario': 5811.8, 'historico_credito': 0.45, 'emprestimos_ativos': 3.0} provavelmente não será inadimplente.
Este cliente: {'idade': 64.0, 'salario': 12258.81, 'historico_credito': 1.0, 'emprestimos_ativos': 2.0} provavelmente não será inadimplente.
Este cliente: {'idade': 41.0, 'salario': 2727.73, 'historico_credito': 0.67, 'emprestimos_ativos': 5.0} provavelmente não será inadimplente.
Este cliente: {'idade': 43.0, 'salario': 7921.23, 'historico_credito': 0.16, 'emprestimos_ativos': 4.0} provavelmente será inadimplente.
Este cliente: {'idade': 42.0, 'salario': 8841.73, 'historico_credito': 1.0, 'emprestimos_ativos': 3.0} provavelmente não será inadimplente.
Este cliente: {'id

In [8]:
# Criar um DataFrame para armazenar os resultados
results = []



In [9]:
# Loop para coletar a previsão para cada cliente
for idx, prediction in enumerate(predicted_classes):
    client_data = data.iloc[idx].to_dict()
    client_data["Previsão"] = "provavelmente não será inadimplente" if prediction == 0 else "provavelmente será inadimplente"
    results.append(client_data)



In [10]:
# Converter a lista de dicionários em um DataFrame
results_df = pd.DataFrame(results)

print('\nPrevisões Salvas em Disco em resultados/resultados_previsoes.csv!\n')




Previsões Salvas em Disco em resultados/resultados_previsoes.csv!



In [11]:
# Salvar o DataFrame em um arquivo CSV
results_df.to_csv("resultados/resultados_previsoes_michelle.csv", index=False)